In [ ]:
# https://docs.cohere.com/v2/reference/rerank

In [6]:
import cohere

co = cohere.ClientV2(api_key = "Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")

docs = [
    "Carson City is the capital city of the American state of Nevada.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
    "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
    "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
]

response = co.rerank(
    model="rerank-v3.5",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)
print(response)

id='b70033c4-d5d1-48c1-930a-3c967bbc2d10' results=[V2RerankResponseResultsItem(document=None, index=3, relevance_score=0.8742601), V2RerankResponseResultsItem(document=None, index=0, relevance_score=0.17292508), V2RerankResponseResultsItem(document=None, index=4, relevance_score=0.10793502)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='2', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)


In [ ]:
# 1. 依赖安装
# pip install langchain langchain-community chromadb cohere

# 2. 文档加载与处理
# from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # 加载PDF文档
# loader = PyPDFLoader("data.pdf")
# documents = loader.load()

# # 文本分割
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200
# )
# chunks = text_splitter.split_documents(documents)

# 3. 向量化与索引构建
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 使用ChatGLM的Embedding模型（需本地部署）
# embeddings = HuggingFaceEmbeddings(
#     model_name="THUDM/chatglm3-6b-32k",
#     model_kwargs={"device": "cuda"}
# )

# # 构建Chroma向量数据库
# vectorstore = Chroma.from_documents(
#     chunks,
#     embeddings,
#     persist_directory="./chroma_db"
# )

# 4. 检索与重排序配置
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
compressor = CohereRerank(
    client=cohere_client,
    top_n=5  # 保留Top5相关文档
)

# 创建带重排序的检索器
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 5. LLM模型初始化
from langchain_community.llms import ChatGLM

# 加载ChatGLM模型（需本地部署）
llm = ChatGLM(
    endpoint_url="http://localhost:8000/v1/chat/completions",
    max_tokens=4096,
    temperature=0.1
)

# 6. 构建RAG链
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 提示模板设计
template = """基于以下上下文，用中文回答用户问题：
上下文：{context}
问题：{question}
要求：答案需包含引用来源的页码（示例：[P3]）"""
prompt = ChatPromptTemplate.from_template(template)

# 组装RAG链
rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 7. 使用示例
response = rag_chain.invoke("请解释量子计算的核心原理？")
print(response)